In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark -q

In [4]:
#Iniciando sessão spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("CVM_Bronze_Ingestion") \
    .getOrCreate()

print("✅ Drive montado e Spark iniciado!")

✅ Drive montado e Spark iniciado!


In [5]:
import zipfile
import os

# 1. Definição dos caminhos
caminho_zip = '/content/drive/MyDrive/pipeline_bigdata_pyspark/dados_cvm_raw.zip'
pasta_bronze = '/content/drive/MyDrive/pipeline_bigdata_pyspark/bronze_raw_data'

# 2. Criar a pasta de destino se não existir
if not os.path.exists(pasta_bronze):
    os.makedirs(pasta_bronze)
    print(f"Diretório criado: {pasta_bronze}")

# 3. Descompactar o arquivo ZIP
with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
    zip_ref.extractall(pasta_bronze)
    print("✅ Arquivos extraídos com sucesso!")

# 4. Ler os dados com Spark
# O asterisco (*) instrui o Spark a ler todos os CSVs da pasta
df_cvm = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", ";") \
    .option("inferSchema", "true") \
    .load(f"{pasta_bronze}/*.csv")

print(f"📊 Total de registros carregados: {df_cvm.count()}")
df_cvm.show(5)

Diretório criado: /content/drive/MyDrive/pipeline_bigdata_pyspark/bronze_raw_data
✅ Arquivos extraídos com sucesso!
📊 Total de registros carregados: 567834
+---------------+------------------+------------+----------+----------+----------+-------------+---------+--------+--------+
|TP_FUNDO_CLASSE| CNPJ_FUNDO_CLASSE|ID_SUBCLASSE| DT_COMPTC|  VL_TOTAL|  VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+---------------+------------------+------------+----------+----------+----------+-------------+---------+--------+--------+
|             FI|00.017.024/0001-53|        NULL|2024-01-02|1136699.13|34.2988597|    1139708.1|      0.0|     0.0|       1|
|             FI|00.017.024/0001-53|        NULL|2024-01-03|1137245.82|34.3123029|    1140154.8|      0.0|     0.0|       1|
|             FI|00.017.024/0001-53|        NULL|2024-01-04|1137741.93|34.3260232|   1140610.71|      0.0|     0.0|       1|
|             FI|00.017.024/0001-53|        NULL|2024-01-05|1138240.64|34.3382208|   1141016.0

In [6]:
#particionamento Parquet e compressão snappy
caminho_bronze_parquet = '/content/drive/MyDrive/pipeline_bigdata_pyspark/bronze/fundos_cvm'

df_cvm.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(caminho_bronze_parquet)

print(f"✅ Camada Bronze finalizada com sucesso!")
print(f"📁 Dados salvos em: {caminho_bronze_parquet}")

# 3. Teste de leitura: Garantir que o Parquet está íntegro
df_teste = spark.read.parquet(caminho_bronze_parquet)
print(f"📊 Verificação: {df_teste.count()} linhas lidas do arquivo Parquet.")

✅ Camada Bronze finalizada com sucesso!
📁 Dados salvos em: /content/drive/MyDrive/pipeline_bigdata_pyspark/bronze/fundos_cvm
📊 Verificação: 567834 linhas lidas do arquivo Parquet.
